In [1]:
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

## Variables

In [2]:
user_name = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')

In [3]:
DB_NAME = 'BacterialGrowth'

## Connexion to the database

In [5]:
cnx = mysql.connector.connect(user=user_name, password=password,host='localhost',database=DB_NAME)

[Cursor documentation](https://www.psycopg.org/docs/cursor.html)

[I think, better documentation for cursor](https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor.html)

In [6]:
# create cursor to read or write
cursor = cnx.cursor()

## Existing tables

In [7]:
pd.read_sql_query('show tables from {}'.format(DB_NAME),cnx)

,Tables_in_bacterialgrowth
0,Bacteria
1,BacteriaCommunity
2,CultivationConditions
3,Experiment
4,Precultivation
5,Reactor
6,TechnicalReplicates


## Insert into tables

### Read file and transform into pandas --> separate by abundance, pH and metabolites --> create files --> save in the DB

#### Form the lists that contain the abundance, pH or metabolites respective columns names

In order to do this we need:
* A file contaning all the unique possible headers that have been found parsing all the provided files.
* The regex expression, which is set manually. For bacterial abundances, fields that contain **time, active, liquid, OD**, but if I find others, like **qPCR, 16S rRNA seq, ...** they must be added in the regex. For pH measurements, only **ph and time**. For metabolites, it considers that everything that was not count on abundances and tme, belongs to metabolites + time.

In [8]:
headersFile = '/Users/julia/bacterialGrowth_thesis/IntermediateFiles/lab_headers.txt'

with open(headersFile) as file:
    headers_list_split = file.read().splitlines()
    headers_list = file.read()
    print(headers_list)

In [9]:
headers_list_split

['Acetate',
 'Formate',
 'Glucose',
 'Lactate',
 'Neu5Ac',
 'OD',
 'Pyruvate',
 'Succinate',
 'liquidactivecount',
 'liquidtotalcount',
 'pH',
 'time']

In [10]:
# Using regex so it is case insensitive and look for patterns inside words (not only complete words)

def getMatchingList (regex, lst):
    list = []
    for word in lst:
        if regex.findall(word):
            list.append(word)
    return list

In [11]:
growth_regex = re.compile(r'.*time.* | .*liquid.* | .*active.* | .*OD.*', flags=re.I | re.X)
growth_regex = re.compile(r'time | liquid | active | OD', flags=re.I | re.X)
ph_regex = re.compile(r'.*time.* | .*ph.*', flags=re.I | re.X)

growth_fields = getMatchingList(growth_regex, headers_list_split)
ph_fields = getMatchingList(ph_regex, headers_list_split)

growth_fields

['OD', 'liquidactivecount', 'liquidtotalcount', 'time']

In [12]:
not_metabolites_list = list(set(growth_fields) | set(ph_fields))
metabolites_fields = set(headers_list_split) - set(not_metabolites_list)
metabolites_fields = list(metabolites_fields)
metabolites_fields.append('time')
metabolites_fields

['Succinate',
 'Formate',
 'Acetate',
 'Lactate',
 'Neu5Ac',
 'Glucose',
 'Pyruvate',
 'time']

In [13]:
headers = {'growth_fields': growth_fields,
          'metabolites_fields': metabolites_fields,
          'ph_fields': ph_fields}

In [14]:
headers['ph_fields']

['pH', 'time']

### Several paths

THINK:
- One options is to call the .py file containing this code with a file containing all the new paths. 
- Make the file with all the paths in the IntermediateFiles directory

Or

- The user will use a first (and hopw, unique) python script in which all the experiment information will be provided. Then it should give the directory with all the files. These files need to be parsed and is needed, for this python code, to get the headers inside all of the files and the paths for saving the separate files for the abundances, metabolites and ph.

In [15]:
# These paths will come from the user!!

files = open("/Users/julia/bacterialGrowth_thesis/IntermediateFiles/listOfFiles.list", "r").readlines()
files = list(map(lambda s: s.strip(), files))

In [16]:
files

['/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/9/BT_batch_mucin1_9.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/7/BT_batch_mucin1_7.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/6/BT_batch_mucin1_6.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/1/BT_batch_mucin1_1.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/8/BT_batch_mucin1_8.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/4/BT_batch_mucin1_4.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/3/BT_batch_mucin1_3.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/2/BT_batch_mucin1_2.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/5/BT_batch_mucin1_5.txt']

In [17]:
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [18]:
for f in files:
    #Get directory of the provided paths
    path_end = max(findOccurrences(f, "/"))
    path = f[:path_end+1]
    
    #Read file with all the data
    df = pd.read_table(f, sep=" ")

    growth_data = df[df.columns.intersection(growth_fields)]
    growth_data = growth_data.round({'OD': 3})
    
    ph_data = df[df.columns.intersection(ph_fields)]
    ph_data = ph_data.round({'pH': 3})

    metabolites_data = df[df.columns.intersection(metabolites_fields)]
    
    # If len(df.columns) <= 1 (only time column), we do not save it
    if len(growth_data.columns) > 1:
        growth_data.to_csv(path+'abundance_file.txt', sep=" ", index=False)
    
    if len(ph_data.columns) > 1:        
        ph_data.to_csv(path+'pH_file.txt', sep=" ", index=False)
    
    if len(metabolites_data.columns) > 1:
        metabolites_data.to_csv(path+'metabolites_file.txt', sep=" ", index=False)

    q = f""" INSERT IGNORE INTO TechnicalReplicates (bacterialAbundanceFile)
        VALUES ('{path+'abundance_file.txt'}')
        """

    cursor.execute(q)
    cnx.commit()
    
    q = f""" INSERT IGNORE INTO TechnicalReplicates (metabolitesFile)
        VALUES ('{path+'metabolites_file.txt'}')
        """

    cursor.execute(q)
    cnx.commit()
    
    #q = f""" INSERT IGNORE INTO FilePath (filePath)
    #    VALUES ('{path+'pH_file.txt'}')
    #    """

    #cursor.execute(q)

cnx.commit()
print('DONE!')

DONE!


In [19]:
path = '/Users/julia/bacterialGrowth_thesis/Data/RI_mucin_RNAseq/1/'

q = f""" INSERT IGNORE INTO TechnicalReplicates (bacterialAbundanceFile)
        VALUES ('/Users/julia/bacterialGrowth_thesis/Data/RI_mucin_RNAseq/1/abundance_file.txt')
        """
cursor.execute(q)
cnx.commit()

In [20]:
for i in pd.read_sql('SELECT bacterialAbundanceFile FROM TechnicalReplicates', cnx):
    print(i)

bacterialAbundanceFile


In [21]:
pd.read_sql('SELECT bacterialAbundanceFile FROM TechnicalReplicates', cnx)

,bacterialAbundanceFile
0,/Users/julia/bacterialGrowth_thesis/Data/BTRI_...
1,None


In [22]:
cursor.close()

cnx.commit()
cnx.close()